In [17]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
import math
from datetime import datetime
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer
import biogeme.biogeme as bio
from biogeme.expressions import Beta, log, Elem, bioNormalCdf
import sys
import biogeme.biogeme as bio
from biogeme import models
import biogeme.exceptions as excep
import biogeme.distributions as dist
import biogeme.results as res
from biogeme.expressions import (
    Beta,
    RandomVariable,
    exp,
    log,
    Integrate,
)

In [18]:
# Import psychometric variables
micro_psych = pd.read_csv('Data/Psych_Micro.csv')
# drop unique_ID column
micro_psych = micro_psych.drop('unique_id', axis=1)
micro_psych = micro_psych.drop('user', axis = 1)
# drop envbenefit4 since unused and contains NaNs
micro_psych = micro_psych.drop('envbenefit4', axis = 1)


In [19]:
# sort
micro_psych = micro_psych.sort_values(by=['who'])

# convert to database
database = db.Database('micro_psych', micro_psych)

# database.panel("who")


globals().update(database.variables)

In [20]:
FILENAME = 'b02_ordered_latent_SN_EP_EB'
try:
    structResults = res.bioResults(pickleFile=f'{FILENAME}.pickle')
except excep.BiogemeError:
    print(
        f'Run first the script {FILENAME}.py in order to generate the '
        f'file {FILENAME}.pickle.'
    )
    sys.exit()
structBetas = structResults.getBetaValues()

In [21]:
intercept_SN = structBetas['intercept_SN']
coef_veh_SN = structBetas['coef_veh_SN']
coef_bike_SN = structBetas['coef_bike_SN']
coef_white_SN = structBetas['coef_white_SN']
coef_higheredu_SN = structBetas['coef_higheredu_SN']
coef_employ_SN = structBetas['coef_employ_SN']

intercept_EP = structBetas['intercept_EP']
coef_veh_EP = structBetas['coef_veh_EP']
coef_bike_EP = structBetas['coef_bike_EP']
coef_stu_EP = structBetas['coef_stu_EP']
coef_higheredu_EP = structBetas['coef_higheredu_EP']

# intercept_RISK = structBetas['intercept_RISK']
# coef_bike_RISK = structBetas['coef_bike_RISK']
# coef_stu_RISK = structBetas['coef_stu_RISK']
# coef_gender_RISK = structBetas['coef_gender_RISK']
# coef_white_RISK = structBetas['coef_white_RISK']
# coef_higheredu_RISK = structBetas['coef_higheredu_RISK']
# coef_employ_RISK = structBetas['coef_employ_RISK']

intercept_EB = structBetas['intercept_EB']
coef_veh_EB = structBetas['coef_veh_EB']
coef_bike_EB = structBetas['coef_bike_EB']
coef_gender_EB = structBetas['coef_gender_EB']

# coef_SN = structBetas['coef_SN']
# coef_EP = structBetas['coef_EP']
# coef_RISK = structBetas['coef_RISK']
# coef_EB = structBetas['coef_EB']


In [22]:
sigma_SN = Beta('sigma_SN', 1, None, None, 0)
sigma_EP = Beta('sigma_EP', 1, None, None, 0)
# sigma_RISK = Beta('sigma_RISK', 1, None, None, 0)
sigma_EB = Beta('sigma_EB', 1, None, None, 0)

In [28]:
# define latent variables
# STRUCTURAL EQUATIONS

SN = (
    intercept_SN
    + coef_veh_SN * veh_dummy
    + coef_bike_SN * bike
    + coef_white_SN * white
    + coef_higheredu_SN * higheredu
    + coef_employ_SN * employ
    + sigma_SN * bioDraws('EC', 'NORMAL_HALTON3')
)

EP = (
    intercept_EP
    + coef_veh_EP * veh_dummy
    + coef_bike_EP * bike
    + coef_stu_EP * stu_dummy
    + coef_higheredu_EP * higheredu
    + sigma_EP * bioDraws('EC', 'NORMAL_HALTON3')
)

# RISK = (
#     intercept_RISK
#     + coef_bike_RISK * bike 
#     + coef_stu_RISK * stu_dummy
#     + coef_gender_RISK * gender_F
#     + coef_white_RISK * white
#     + coef_higheredu_RISK * higheredu
#     + coef_employ_RISK * employ
#     # + sigma_ATT * omega_ATT
# )

EB = (
    intercept_EB
    + coef_veh_EB * veh
    + coef_bike_EB * bike
    + coef_gender_EB * gender_F
    + sigma_EB * bioDraws('EC', 'NORMAL_HALTON3')
)

# INTENT = (
#     + coef_SN*SN
#     + coef_EP * EP
#     + coef_RISK * RISK
#     + coef_EB * EB
# )


In [29]:
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 1)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_SCOOTERTIME_PRIME = Beta('B_SCOOTERTIME_PRIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_PRIME = Beta('B_COST_PRIME', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ = Beta('B_AGE_SQ', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F = Beta('B_GENDER_F', 0, None, None, 0)

B_AGE_MORE_35 = Beta('B_AGE_MORE_35', 0, None, None, 0)
B_CHILD_IN_HH = Beta('B_CHILD_IN_HH', 0, None, None, 0)
B_IS_WHITE = Beta('B_IS_WHITE', 0, None, None, 0)
B_FULLTIME_EMPLOY = Beta('B_FULLTIME_EMPLOY', 0, None, None, 0)
B_HHINCOME_MORE_75k = Beta('B_HHINCOME_MORE_75k', 0, None, None, 0)

B_EP_DKBIKE = Beta('B_EP_DKBIKE', 0, None, None, 0)
B_EB_DKBIKE = Beta('B_EP_DKBIKE', 0, None, None, 0)
B_SN_DKBIKE = Beta('B_SN_DKBIKE', 0, None, None, 0)

B_EP_DLBIKE = Beta('B_EP_DLBIKE', 0, None, None, 0)
B_EB_DLBIKE = Beta('B_EP_DLBIKE', 0, None, None, 0)
B_SN_DLBIKE = Beta('B_SN_DLBIKE', 0, None, None, 0)

B_EP_SCTRANS = Beta('B_EP_SCTRANS', 0, None, None, 0)
B_ATT_SCTRANS = Beta('B_ATT_SCTRANS', 0, None, None, 0)
B_SN_SCTRANS = Beta('B_SN_SCTRANS', 0, None, None, 0)
B_EB_SCTRANS = Beta('B_EB_SCTRANS', 0, None, None, 0)

B_INTENT_BIKE = Beta('B_INTENT_BIKE', 0, None, None, 0)
B_INTENT_SCTRANS = Beta('B_INTENT_SCTRANS', 0, None, None, 0)
B_INTENT_SCOOT = Beta('B_INTENT_SCOOT', 0, None, None, 0)



In [30]:
# Choice model
V7 = (ASC_SCOOTER #_RND
      + B_SCOOTERTIME*sctime
      + B_COST*sccost_adj
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech
#     + ASC_SCOOTER_PRIME*AVtech
#     + B_SCOOTERTIME_PRIME*sctime*AVtech
#     + B_COST_PRIME*sccost_adj*AVtech
      )

V8 = (ASC_DLBIKE #_RND
      + B_BIKETIME*dltime
      + B_COST*dlcost_adj
      + B_ACCESS*DLAW
      + B_SN_DLBIKE*SN
      + B_EP_DLBIKE*EP
      + B_EB_DLBIKE*EB     
    )

V9 = (ASC_DKBIKE #_RND 
      + B_BIKETIME*dbtime 
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_SN_DKBIKE*SN
      + B_EP_DKBIKE*EP
      + B_EB_DKBIKE*EB     
    )

V10 = (ASC_SCTRANSIT #_RND
       + B_SCTRANSITTIME*sttotime
       + B_COST*sttocost_adj 
       + B_ACCESS*STAW*(1-AVtech)
       + B_WAITAV*STAV*AVtech
       + B_SN_SCTRANS*SN
       + B_EP_SCTRANS*EP
       + B_EB_SCTRANS*EB     
)


In [31]:
V = {7: V7, 8: V8, 9: V9, 10: V10}
av = {7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}
      

############################################### DO WE NEED SEPARATE COND PROB?
condprob = models.logit(V, av, choice)
# condlike = (
#     P_ATT01 * 
#     P_ATT02 *
#     P_ATT03 *
#     P_EP01 *
#     P_EP02 *
#     P_EP03 *
#     P_EP04 *
#     P_EP05 *
#     P_SN01 *
#     P_SN02 *
#     P_SN03 *
#     condprob
# )
loglike = log(MonteCarlo(condprob))

# condlikeindiv = PanelLikelihoodTrajectory(condlike)
# condprobindiv = PanelLikelihoodTrajectory(condprob)
# logprob = log(MonteCarlo(condlikeindiv))

In [32]:

the_biogeme = bio.BIOGEME(database, loglike)
the_biogeme.modelName = "b04_latent_choice_seq_SN_EP_EB"

# Estimate the parameters
results = the_biogeme.estimate()
print(f"Estimated betas: {len(results.data.betaValues)}")
print(f"Final log likelihood: {results.data.logLike:.3f}")

[17:54:37] < Warning >   Cannot read file __b04_latent_choice_seq_SN_EP_EB.iter. Statement is ignored.
